### Callbacks / Event-Handlers
 
- In Python sind **Funktionen** wie man sagt **first class citizens**. Funktionen sind Objekte wie alle anderen und k&ouml;nnen u.a. Variabeln zugewiesen werden oder (anderen) Funktionen als Argumente &uuml;bergeben werden.  

- Eine Funktion, welche einer anderen Funktion als Argument &uuml;bergeben wird, wird **Callback** genannt.
Oft wird diese Funktion beim Eintreffen eines sog. **Events** 
(Tastendruck, Mausklick, Mausklick auf Button, Ausw&auml;hlen einer Option in einem Menu,...) aufgerufen und dann auch **Event-Handler** genannt.  

- Das Sicherstellen, dass der Callback 
zu einem sp&auml;teren Zeitpunkt z.B. von einem Button-Objekt aufgerufen wird, nennt sich auch **Registrierung des Callbacks**.

***
**Ein erstes Beispiel**
***

In [ ]:
def str2list(s, f):
    '''trenne den String s beim ',' -> Liste
       wende f auf Listenelemente an
    '''   
    lst = s.split(',')
    return [f(x) for x in lst]

# wird als Callback verwendet
def double(x):
    return x + x

In [ ]:
s = '1,2,3,4,5'
str2list(s, double)

***
Anstelle einer Funktion kann auch ein Objekt `obj` f&uuml;r welches
`obj(x)` syntaktisch korrekter Code ist (ein **Callable**) &uuml;bergeben werden.
***

In [ ]:
s = '1,2,3,4,5'
str2list(s, int)
# str2list(s, list)

### Ein Dictionary mit Callbacks
Wir wollen Elemente einer Liste bearbeiten.
Je nach Typ des Listenelements soll etwas mit dem Element gemacht werden.  

Wir benutzen einen Dictionary `callbacks`. 
F&uuml;r einen Typ (z.b. `int`) ist
`callbacks[int]` z.B. eine **Liste** mit Funktionen, welche
der Reihe nach zum Modifizieren der Listenelemente dieses Typs verwendet werden sollen.
Z.B. Integers sollen zuerst quadriert und dann in einen String umgewandelt werden.

In [ ]:
def pick_and_apply_callbacks(item, callbacks):
    '''wende die Funktionen in callbacks[type(item)] 
       der Reihe nach auf item an
       
       gib item unveraendert zurueck falls type(item) kein Key im dict callbacks
    '''
    funs = callbacks.get(type(item), [])
    for fun in funs:
        item = fun(item)
    return item  

def process_list(lst, callbacks):
    '''modifiziere die Elemente der Liste lst
       mit den Funktionen im dict callbacks 
    '''
    return [pick_and_apply_callbacks(item, callbacks) for item in lst]

def square(x):
    return x**2

In [ ]:
lst = [1, 2, 3, 'foo']
callbacks = {int: [square, str]}
process_list(lst, callbacks)


### Callbacks verwalten
Die Funktion   
`register_callback(typename, fun, remove = False)`  
erlaubt das Hinzuf&uuml;gen und Entfernen von Callbacks.   

**Die Registrierung von Callbacks bei Widgets in Jupyterlab folgt im Wesentlichen diesem Muster.**

In [ ]:
# Dictionary zum Speichern der Callbacks
callbacks = {}

# Funktion zum Verwalten der Callbacks
def register_callback(typename, fun, remove = False):
    '''registriere fun als Callback fuer den Typ typename
       
       Ist remove = True, wird der Callback entfernt 
    '''
    if remove:
        if fun in callbacks[typename]:
            callbacks[typename].remove(fun)
            print('unregistered callback {} for type {}'\
                  .format(fun.__name__, typename.__name__)
                 )
        
    elif fun in callbacks.get(typename, []):
        print('Callback {} already registered for type {}'\
              .format(fun.__name__, typename.__name__)
             )
        
    else:
        callbacks.setdefault(typename,[]).append(fun)
        print('Callback {} registered for type {}'\
              .format(fun.__name__,typename.__name__))

***
**Beachte:**  
Wird nachfolgende Zelle ein zweites Mal ausgef&uuml;hrt, so
werden neue Funktionen `square` und `round_1` erzeugt.  
In der n&auml;chsten Zelle werden dann diese Funktionen
als Callbacks registriert, ohne die alten Funktionen zu entfernen.  

Auf die alten Funktionen kann nun nicht mehr via die Variabelnamen 
`square` und `round_1` zugegriffen werden.
Das **verunm&ouml;glicht das Entfernen** der alten Funktionen als Callbacks mit unserer Funktion `register_callback`.
***

In [ ]:
# callbacks
def square(x):
    return x**2
def round_1(x):
    return round(x, 1)

In [ ]:
# callbacks registrieren
# Integer verdoppeln und dann in String umwandeln
# floats auf eine Kommastelle runden
# Bei Strings Methode capitalize anwenden 
# (ersten Buchstaben gross machen, Rest klein)

callbacks = {}
register_callback(int, square)
register_callback(int, str)
register_callback(float, round_1)
register_callback(str, str.capitalize)

In [ ]:
# Funktionsnamen und id (Addresse des Objekts im Speicher) 
# der Callbacks im Dictionary callbacks anzeigen
{k: [(callback.__name__, id(callback)) for callback in v]\
    for k, v in callbacks.items()
}

In [ ]:
# Listenelemente modifizieren
data = [2, 2.23, 'foo']
process_list(data, callbacks)

In [ ]:
register_callback(int, square,remove = True)